In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import inception_v3
from IPython.display import Image, display
from tqdm import tqdm
import re

In [2]:
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [3]:
def preprocess_image(img):
    """ Preprocess image for inception_v3 """
    
    img = keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = inception_v3.preprocess_input(img)
    
    return img

def deprocess_image(x):
    """ Deprocess image from inception_v3 tensor """
    
    x = x.reshape((x.shape[1], x.shape[2], 3))
    # Undo inception_v3 preprocess
    x /= 2.
    x += 0.5
    x *= 255

    x = np.clip(x, 0, 255).astype('uint8')
    
    return x

In [5]:
def compute_loss(input_image, feature_extractor, layer_settings):
    """
    Computes loss based on a feature extractor and the corresponding layers weights.
    
    Parameters:
        input_image (np.ndarray): numpy array of an image.
        feature_extractor (tf.Model): feature extractor with outputs corresponding to the layers.
        layer_settings (dict): dict with tf model layer names and corresponding weights in the loss function.
        
    Returns:
        loss (float): loss of the image w.r.t. the layers specified in layer_settings.
    """
    
    # Computes features from image
    features = feature_extractor(input_image)
    
    loss = tf.zeros(shape=())
    for name in features.keys():
        # Coefficient of the layer
        coeff = layer_settings[name]
        # Activation of the layer
        activation = features[name]
        # Adds to loss (avoid artifacts by removing borders)
        scaling = tf.reduce_prod(tf.cast(tf.shape(activation), 'float32'))
        #loss += coeff * tf.reduce_sum(tf.square(activation[:,2:-2,2:-2,:])) / scaling
        loss += coeff * tf.reduce_sum(tf.square(activation)) / scaling
    
    return loss

@tf.function
def gradient_ascent_step(img, feature_extractor, layer_settings, learning_rate):
    """
    Performs a gradient ascent step on an image.
    
    Parameters:
        img (np.ndarray): numpy array of an image.
        feature_extractor (tf.Model): feature extractor with outputs corresponding to the layers.
        layer_settings (dict): dict with tf model layer names and corresponding weights in the loss function.
        learning_rate (float): learning rate for the gradient ascent step.
        
    Returns:
        loss (float32): loss of the image w.r.t. the layers specified in layer_settings.
        img (np.ndarray): numpy array of the modified image
    """
    
    # Computes loss with GradientTape
    with tf.GradientTape() as tape:
        tape.watch(img)
        loss = compute_loss(img, feature_extractor, layer_settings)
    # Computes gradients and normalize
    grads = tape.gradient(loss, img)
    grads /= tf.maximum(tf.reduce_mean(tf.abs(grads)), 1e-6)
    # Gradient ascent step
    img += learning_rate * grads
    
    return loss, img

def gradient_ascent_loop(img, feature_extractor, layer_settings, iterations, learning_rate, octave, shape, max_loss=None):
    """
    Performs the gradient ascent loop on an image.
    
    Parameters:
        img (np.ndarray): numpy array of an image.
        feature_extractor (tf.Model): feature extractor with outputs corresponding to the layers.
        layer_settings (dict): dict with tf model layer names and corresponding weights in the loss function.
        iterations (int): number of iterations for the loop.
        learning_rate (float): learning rate for the gradient ascent steps.
        max_loss (float): maximum loss before interruption (default=None).
        
    Returns:
        img (np.ndarray): numpy array of the modified image
    """
    
    # gradient ascent loop
    losses = []
    t = tqdm(range(iterations))
    for i in t:
        loss, img = gradient_ascent_step(img, feature_extractor, layer_settings, learning_rate)
        losses.append(loss)
        
        t.set_description("Octave: %d, Shape: %s, Loss: [%.2f, %.2f]" % (octave, shape, np.min(losses), np.max(losses)))
        t.refresh()
        if max_loss is not None and loss > max_loss:
            break
        #print("Loss at step %d: %.2f" % (i, loss))
    #print("Min loss: %.2f - Max loss: %.2f" % (np.min(losses), np.max(losses)))
    return img

In [6]:
def dreamify(image_path,
             source_filename,
             destination_filename,
             model,
             layer_settings,
             learning_rate = 0.01,
             num_octave = 3,
             octave_scale = 1.5,
             iterations = 20,
             max_loss = 15.):
    """
    Dreamifies an image.
    Returns nothing, the image is automatically saved at the requested destination.
    
    Parameters:
        image_path (str): path to the image folder (must end with "/").
        source_filename (str): name of the original file (with file format).
        destination_filename (str): name of the destination file (without file format).
        model (tf.Model): model to be used.
        layer_settings (dict): dict with tf model layer names and corresponding weights in the loss function.
        learning_rate (float): learning rate for the gradient ascent steps (deault=0.01).
        num_octave (int): number of subsampling octaves (default=3).
        octave_scale (float): scale of each subsampling (deault=1.5).
        iterations (int): number of iterations for the gradient ascent loop at each scale (deafault=20).
        max_loss (float): maximum loss before interruption on a loop (default=15.0).
    """
    
    # Dict of output layers
    outputs_dict = dict([(layer.name, layer.output)
                         for layer in [model.get_layer(name)
                                       for name in layer_settings.keys()]])
    # Feature extractor from model input layer and dict of output layers
    feature_extractor = keras.Model(inputs = model.inputs,
                                    outputs = outputs_dict)
    
    # Loads image, preprocesses it and gets its shape
    original_img = keras.preprocessing.image.load_img(image_path + source_filename)
    original_img = preprocess_image(original_img)
    original_shape = original_img.shape[1:3]
    
    # Creates the list of successive shapes to use
    successive_shapes = [original_shape]
    for i in range(1, num_octave):
        shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
        successive_shapes.append(shape)
    successive_shapes = successive_shapes[::-1] # Invert
    
    # Image at minimum shape
    shrunk_original_img = tf.image.resize(original_img, successive_shapes[0])
    # Image to be modified
    img = tf.identity(original_img)
    
    for i, shape in enumerate(successive_shapes):
        #print("Octave %d with shape %s" % (i, shape))
        # Resizes at current shape
        img = tf.image.resize(img, shape)
        # Performs the gradient ascent loop on the current shape
        img = gradient_ascent_loop(img, 
                                   feature_extractor, 
                                   layer_settings,
                                   iterations=iterations,
                                   learning_rate=learning_rate,
                                   octave=i,
                                   shape=shape,
                                   max_loss=max_loss)
        # Restores lost details
        upscaled_shrunk_original_img = tf.image.resize(shrunk_original_img, shape)
        same_size_original = tf.image.resize(original_img, shape)
        lost_detail = same_size_original - upscaled_shrunk_original_img
        img += lost_detail
        # Resizes the minimum shape to the current shape for next loop
        shrunk_original_img = tf.image.resize(original_img, shape)
        
    # Stores the resulting image
    keras.preprocessing.image.save_img(image_path + destination_filename + '.png', deprocess_image(img.numpy()))

In [7]:
def randomize_layer_settings(layer_names, coeffs_sum=15, seed=0):
    """ 
    Creates a randomized layer settings dictionary.
    
    Parameters:
        layer_names ([str]): list of layers names, will be used as keys of the dict.
        coeffs_sum (float): sum of the requested coefficients (default=15).
        seed (int): seed for the random sampling (default=0).
        
    Returns:
        layer_settings (dict): layer coefficients as a dict.
    """
    
    np.random.seed(seed)
    
    layer_settings = {name : np.random.uniform() for name in layer_names}
    
    scale_factor = coeffs_sum / np.sum(list(layer_settings.values()))
    
    layer_settings = {name : coeff * scale_factor for (name, coeff) in layer_settings.items()}
    
    return layer_settings

In [8]:
def pair_to_string(key, value, decimals=2):
    """
    Turns a dict pair into a string for naming purposes.
    
    Parameters:
        key (str): layer name.
        value (float): layer coefficient.
        decimals (int): number of decimals to be used in np.around (default=2).
        
    Returns:
        output (str): the resulting string.
    """
    
    initial_letter = key[0]
    key = re.search('[0-9][a-z0-9]*', key)
    
    return initial_letter + key.group(0) + "=" + str(np.around(value, decimals=decimals))

In [10]:
layer_names = ['mixed' + str(i) for i in range(0,4)]
layer_settings = randomize_layer_settings(layer_names, coeffs_sum=7)

image_path = "Images/InsertAlbumHere/"
source_filename = "original.jpg"
destination_filename = "-".join([pair_to_string(k,v) for (k,v) in layer_settings.items()])

# decomment before running
#dreamify(image_path, source_filename, destination_filename, model, layer_settings)
#
#display(Image(image_path + destination_filename + '.png'))